In [1]:
import asyncio
import random
from collections import defaultdict
from dataclasses import dataclass
from typing import Dict, List

from autogen_core.application import WorkerAgentRuntime, WorkerAgentRuntimeHost
from autogen_core.base import MessageContext
from autogen_core.base._serialization import try_get_known_serializers_for_type
from autogen_core.components import (
    DefaultTopicId,
    RoutedAgent,
    default_subscription,
    message_handler,
)
from autogen_core.components._default_topic import DefaultTopicId


@dataclass
class GameAction:
    player_name: str
    action: str # Could be "rock", "paper" or "scissors"

@dataclass
class NewRound:
    """Moderator sends this message to ask players to play at the same time"""
    round_id: int


@default_subscription
class GameModerator(RoutedAgent):
    def __init__(self, name: str, num_players: int, max_rounds: int =3 ) -> None:
        super().__init__("A Rock, Paper, Scissors game moderator")
        self.name:str = name
        self._num_players:int = num_players
        self._this_round_actions: List[GameAction] = []
        self._num_played_rounds:int = 0
        self._max_rounds:int = max_rounds
        self._leaderboard: Dict[str, int] = defaultdict()

    @message_handler
    async def on_new_message(self, message: GameAction, ctx: MessageContext) -> None:
        self._this_round_actions.append(message)
        if len(self._this_round_actions) == self._num_players: # Time to see which agent has won?
            winner = self.get_winner()
            print(f"Winner: {winner}\n","-" * 50)
            self._this_round_actions = []
            self._num_played_rounds += 1

            if self._num_played_rounds < self._max_rounds:
                print(f"Starting round {self._num_played_rounds + 1}")
                await asyncio.sleep(2)
                await self.publish_message(NewRound(self._num_played_rounds), DefaultTopicId())
            else:
                print("="*50 + "\nFinal Results\n" + "=" * 50)
                self._print_leaderboard(self._leaderboard)

    def _wins(self, action1: str, action2: str) -> int:
        """Compares two actions to decide the winner"""
        win_conditions = {
            "rock": "scissors",
            "scissors": "paper",
            "paper": "rock"
        }

        if action1 == action2:
            return 0  # Draw
        elif win_conditions[action1] == action2:
            return 1  # action1 wins
        else:
            return -1  # action2 wins

    def get_winner(self) -> str:
        """This function checks the winner of this round and updates the scoreboard"""
        # Assume the first player is the current winner
        current_winner = self._this_round_actions[0]
        draw = False

        # Iterate through other players and compare their actions
        for i in range(1, len(self._this_round_actions)):
            res = self._wins(current_winner.action, self._this_round_actions[i].action)

            if res == 0:  # If it's a draw, mark it
                draw = True
            elif res == -1:  # If the current player loses, update the winner
                current_winner = self._this_round_actions[i]
                draw = False  # Reset the draw status since we have a new potential winner

        # Return draw if applicable or the name of the winning player
        if draw:
            res= "draw"
        else:
            res= current_winner.player_name
            self._update_scoreboard(res)
        return res

    def _update_scoreboard(self, winner:str) -> None:
        if winner in self._leaderboard:
            self._leaderboard[winner] += 1
        else:
            self._leaderboard[winner] = 1

    def _print_leaderboard(self, win_counts: Dict[str, int]) -> None:
        # Sort the dictionary by win count in descending order
        sorted_leaderboard = sorted(win_counts.items(), key=lambda item: item[1], reverse=True)

        print("Leaderboard:")
        for rank, (player, wins) in enumerate(sorted_leaderboard, start=1):
            print(f"{rank}. {player}: {wins} wins")

@default_subscription
class RockAgent(RoutedAgent):
    """This agent always plays rock ... strange strategy ..."""
    def __init__(self, name: str) -> None:
        super().__init__("A Player that always plays rock")
        self.name = name

    @message_handler
    async def on_new_message(self, message: NewRound, ctx: MessageContext) -> None:
        action = "rock"
        print(f"{self.name}@round {message.round_id}: Playing {action}")
        return await self.publish_message(GameAction(self.name, action), DefaultTopicId())


@default_subscription
class RandomAgent(RoutedAgent):
    """This agent takes a random action"""
    def __init__(self, name: str) -> None:
        super().__init__("A Player that always plays rock")
        self.name = name

    @message_handler
    async def on_new_message(self, message: NewRound, ctx: MessageContext) -> None:
        random_action = random.choice(["rock","paper","scissors"])
        print(f"{self.name}@round {message.round_id}: Playing {random_action}")
        return await self.publish_message(GameAction(self.name, random_action), DefaultTopicId())


serializers = [try_get_known_serializers_for_type(GameAction), try_get_known_serializers_for_type(NewRound)]

In [2]:
host_address = "localhost:50060"
host = WorkerAgentRuntimeHost(address=host_address)
host.start()
print(f"Distributed Host is now running and listening for connection at {host_address}")

Distributed Host is now running and listening for connection at localhost:50060


In [3]:
player1 = WorkerAgentRuntime(host_address=host_address)
player1.add_message_serializer(serializers)
player1.start()
await RockAgent.register(player1, "player1", lambda: RockAgent("player 1"))
print("Player 1 joined host")

Player 1 joined host


player 1@round 0: Playing rock
player 1@round 1: Playing rock
player 1@round 2: Playing rock


In [4]:
player2 = WorkerAgentRuntime(host_address=host_address)
player2.add_message_serializer(serializers)
player2.start()
await RandomAgent.register(player2, "player2", lambda: RandomAgent("player 2"))
print("Player 2 joined host")

Player 2 joined host


player 2@round 0: Playing rock
player 2@round 1: Playing rock
player 2@round 2: Playing rock


In [5]:
player3 = WorkerAgentRuntime(host_address=host_address)
player3.add_message_serializer(serializers)
player3.start()
await RandomAgent.register(player3, "player3", lambda: RandomAgent("player 3"))
print("Player 3 joined host")

Player 3 joined host


player 3@round 0: Playing rock
player 3@round 1: Playing paper
player 3@round 2: Playing paper


In [6]:
moderator = WorkerAgentRuntime(host_address=host_address)
moderator.add_message_serializer(serializers)
moderator.start()
await GameModerator.register(moderator, "moderator", lambda: GameModerator("Moderator", num_players= 3, max_rounds=3))
print("Moderator is ready to start the game ...\nStarting round 0")
await asyncio.sleep(3)
await moderator.publish_message(NewRound(0), DefaultTopicId())

Moderator is ready to start the game ...
Starting round 0


Winner: draw
 --------------------------------------------------
Starting round 2
Winner: player 3
 --------------------------------------------------
Starting round 3
Winner: player 3
 --------------------------------------------------
Final Results
Leaderboard:
1. player 3: 2 wins


In [ ]:
# Commenting to avoid accidental run, uncomment and stop
# await host.stop()